# Monitoring setup for Bicycle Sharing Demand Prediction

This notebook shows how you can use the Evidently to:
* calculate performance and data drift for the model, performed as batch checks 
* log models quality & data drift using MLflow Tracking
* explore the result 

More examples are available in the github: https://github.com/evidentlyai/evidently/tree/main/examples

Evidently docs: https://docs.evidentlyai.com/

Join our Discord: https://discord.com/invite/xZjKRaNp8b

In [1]:
try:
    import evidently
except:
    !pip install git+https://github.com/evidentlyai/evidently.git

  Cloning https://github.com/evidentlyai/evidently.git to /tmp/pip-req-build-4b5lj5y5
  Running command git clone --filter=blob:none --quiet https://github.com/evidentlyai/evidently.git /tmp/pip-req-build-4b5lj5y5
  Resolved https://github.com/evidentlyai/evidently.git to commit 2f2900fad39702f284bcbfe8353bf7ba758952e4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 66.7 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached statsmodels-0.14.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.2/395.2 KB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.3/511.3 KB 58.6 MB/s eta 0:00:00
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 KB 33.6 MB/s eta 0:00:00
  Using cached iterative_telemetry-0.0.8-py3-none-any.whl (10 kB)
  Using cached dynaconf-3.2.4-py2.py3-none-any.whl (223 kB)
  Usi

In [5]:
import datetime
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import json

from sklearn import datasets, ensemble, model_selection
from scipy.stats import anderson_ksamp

from evidently.metrics import RegressionQualityMetric, RegressionErrorPlot, RegressionErrorDistribution
from evidently.metric_preset import DataDriftPreset, RegressionPreset
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report

In [6]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Bicycle Demand Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [36]:
content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip", verify=False).content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday']) 


In [45]:
raw_data.iloc[15, :].to_dict()

{'instant': 16,
 'dteday': Timestamp('2011-01-01 00:00:00'),
 'season': 1,
 'yr': 0,
 'mnth': 1,
 'hr': 15,
 'holiday': 0,
 'weekday': 6,
 'workingday': 0,
 'weathersit': 2,
 'temp': 0.44,
 'atemp': 0.4394,
 'hum': 0.77,
 'windspeed': 0.2985,
 'casual': 40,
 'registered': 70,
 'cnt': 110}

In [11]:
raw_data.index = raw_data.apply(lambda row: datetime.datetime.combine(row.dteday.date(), datetime.time(row.hr)),
                                axis=1)

In [33]:
raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
2011-01-01 00:00:00,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01 01:00:00,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01 02:00:00,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01 03:00:00,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01 04:00:00,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [34]:
raw_data[0]

KeyError: 0

## Model training 

In [14]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]#'weathersit']

In [15]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [16]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    reference[numerical_features + categorical_features],
    reference[target],
    test_size=0.3
)

In [17]:
regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)

In [18]:
regressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50, random_state=0)

In [19]:
preds_train = regressor.predict(X_train)
preds_test = regressor.predict(X_test)

## Model validation

In [21]:
X_train['target'] = y_train
X_train['prediction'] = preds_train

X_test['target'] = y_test
X_test['prediction'] = preds_test

In [23]:
column_mapping = ColumnMapping()

column_mapping.target = 'target'
column_mapping.prediction = 'prediction'
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [27]:
regression_performance_report = Report(metrics=[
    RegressionPreset(),
])

regression_performance_report.run(reference_data=X_train.sort_index(), current_data=X_test.sort_index(),
                                  column_mapping=column_mapping)
regression_performance_report.save_html("regression_preset.html")

## Production model training

In [28]:
regressor.fit(reference[numerical_features + categorical_features], reference[target])

RandomForestRegressor(n_estimators=50, random_state=0)

In [29]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [30]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
reference['prediction'] = ref_prediction

In [31]:
regression_performance_report = Report(metrics=[
    RegressionPreset(),
])

regression_performance_report.run(reference_data=None, current_data=reference,
                                  column_mapping=column_mapping)
regression_performance_report.save_html("reg_production_model.html")

## Some time has passed - how is the model working?

In [ ]:
current_prediction = regressor.predict(current[numerical_features + categorical_features])
current['prediction'] = current_prediction

### Week 1

In [ ]:
regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'],
                                  column_mapping=column_mapping)
regression_performance_report

### Week 2

In [ ]:
regression_performance_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution()
])

regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-02-07 00:00:00':'2011-02-14 23:00:00'], 
                                            column_mapping=column_mapping)
regression_performance_report.show()

### Week 3

In [ ]:
regression_performance_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution()
])

regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'], 
                                            column_mapping=column_mapping)
regression_performance_report.show()

## Why the quality has dropped?

In [ ]:
column_mapping_drift = ColumnMapping()

column_mapping_drift.target = target
column_mapping_drift.prediction = prediction
column_mapping_drift.numerical_features = numerical_features
column_mapping_drift.categorical_features = []

In [ ]:
data_drift_report = Report(metrics=[
    DataDriftPreset(),
])

data_drift_report.run(
    reference_data=reference,
    current_data=current.loc['2011-02-14 00:00:00':'2011-02-21 23:00:00'],
    column_mapping=column_mapping_drift,
)
data_drift_report

## Let's customize the report!

In [ ]:
from evidently.core import ColumnType
from evidently.calculations.stattests import StatTest, register_stattest

def _anderson_stat_test(reference_data: pd.Series, current_data: pd.Series, feature_type: ColumnType, threshold: float):
    p_value = anderson_ksamp([reference_data, current_data])[2]
    return p_value, p_value < threshold

anderson_stat_test = StatTest(
    name="anderson",
    display_name="Anderson test (p_value)",
    allowed_feature_types=[ColumnType.Numerical]
)

register_stattest(anderson_stat_test, default_impl=_anderson_stat_test)
# options = DataDriftOptions(feature_stattest_func=anderson_stat_test, all_features_threshold=0.9, nbinsx=20)

In [ ]:
the_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution(),
    DataDriftPreset(stattest=anderson_stat_test, stattest_threshold=0.9),
])


the_report.run(
    reference_data=reference,
    current_data=current.loc['2011-02-14 00:00:00':'2011-02-21 23:00:00'], 
    column_mapping=column_mapping_drift
)
the_report

## Jupyter vs Automated Run?

To run this part of the tutorial you need to install MLflow (if not installed yet)

You can install MLflow with the following command: `pip install mlflow` or install MLflow with scikit-learn via `pip install mlflow[extras]`

More details:https://mlflow.org/docs/latest/tutorials-and-examples/tutorial.html#id5

In [ ]:
try:
    import mlflow
except:
    !pip install mlflow

In [ ]:
import mlflow
#import mlflow.sklearn
from mlflow.tracking import MlflowClient

In [ ]:
experiment_batches = [
    ('2011-01-29 00:00:00','2011-02-07 23:00:00'),
    ('2011-02-07 00:00:00','2011-02-14 23:00:00'),
    ('2011-02-15 00:00:00','2011-02-21 23:00:00'),  
]

In [ ]:
the_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution(),
    DataDriftPreset(stattest=anderson_stat_test, stattest_threshold=0.9),
])

the_report.run(
    reference_data=reference, 
    current_data=current.loc[experiment_batches[0][0]:experiment_batches[0][1]],
    column_mapping=column_mapping_drift
)

In [ ]:
logged_json = json.loads(the_report.json())

In [ ]:
logged_json

In [ ]:
[x['metric'] for x in logged_json['metrics']]

In [ ]:
logged_json['metrics'][0]['result']['current']['mean_error']

In [ ]:
logged_json['metrics'][3]['result']['drift_share']

In [ ]:
#log into MLflow
client = MlflowClient()

#set experiment
mlflow.set_experiment('Model Quality Evaluation')

#start new run
for date in experiment_batches:
    with mlflow.start_run() as run: #inside brackets run_name='test'
        
        # Log parameters
        mlflow.log_param("begin", date[0])
        mlflow.log_param("end", date[1])

        # Get metrics
        the_report = Report(metrics=[
            RegressionQualityMetric(),
            RegressionErrorPlot(),
            RegressionErrorDistribution(),
            DataDriftPreset(stattest=anderson_stat_test, stattest_threshold=0.9),
        ])
        the_report.run(
            reference_data=reference, 
            current_data=current.loc[date[0]:date[1]],
            column_mapping=column_mapping_drift)
        logged_json = json.loads(the_report.json())
        
        me = logged_json['metrics'][0]['result']['current']['mean_error']
        mae = logged_json['metrics'][0]['result']['current']["mean_abs_error"]
        drift_share = logged_json['metrics'][3]['result']['drift_share']
        
        # Log metrics
        mlflow.log_metric('me', round(me, 3))
        mlflow.log_metric('mae', round(mae, 3))
        mlflow.log_metric('drift_share', round(drift_share, 3))

        print(run.info)

In [ ]:
#run MLflow UI (NOT recommended! It will be more convinient to run it directly from the TERMINAL)
#!mlflow ui

# Support Evidently
Enjoyed the tutorial? Star Evidently on GitHub to contribute back! This helps us continue creating free open-source tools for the community. https://github.com/evidentlyai/evidently